In [2]:
from os import name
from sys import modules
import pickle
import copy

from hypernets.core.ops import HyperInput, ModuleChoice
from hypernets.core.search_space import Choice, HyperNode, HyperSpace, ModuleSpace
from hypernets.model import Estimator, HyperModel
from hypernets.utils import fs
from hypernets.searchers.random_searcher import RandomSearcher

from sklearn import neighbors

In [6]:
class Param_space(object):
    def __init__(self, **kwargs):
        super(Param_space, self).__init__()

    @property
    def knn(self):
        return dict(
            cls=neighbors.KNeighborsClassifier,
            n_neighbors=Choice([2, 3, 5, 6]),
            weights=Choice(['uniform', 'distance']),
            algorithm=Choice(['auto', 'ball_tree', 'kd_tree', 'brute']),
            leaf_size=Choice([20, 30, 40]),
            p=Choice([1, 2]),
        )    

    def __call__(self, *args, **kwargs):
        space = HyperSpace()
        with space.as_default():
            hyper_input = HyperInput(name='input')
            model = self.knn
            modules = [ModuleSpace(name=f'{model["cls"].__name__}', **model)]
            outputs = ModuleChoice(modules)(hyper_input)
            space.set_inputs(hyper_input)

        return space

In [31]:
#check the search_space
search_space = Param_space()
searcher = RandomSearcher(search_space, optimize_direction='max')
searcher.sample()

key,value
signature,b4c8e43952e0f0ccea1a662bd2036f88
vectors,"[2, 1, 3, 1, 0]"
0-KNeighborsClassifier.n_neighbors,5
1-KNeighborsClassifier.weights,distance
2-KNeighborsClassifier.algorithm,brute
3-KNeighborsClassifier.leaf_size,30
4-KNeighborsClassifier.p,1


In [32]:
class KnnEstimator(Estimator):
    def __init__(self, space_sample, task='binary'):
        super(KnnEstimator, self).__init__(space_sample, task)

        out = space_sample.get_outputs()[0]
        kwargs = out.param_values
        kwargs = {key: value for key, value in kwargs.items() if not isinstance(value, HyperNode)}

        cls = kwargs.pop('cls')
        self.model = cls(**kwargs)
        self.cls = cls
        self.model_args = kwargs
    
    def fit(self, X, y, **kwargs):
        self.model.fit(X, y, **kwargs)

        return self
    
    def predict(self, X, **kwargs):
        pred = self.model.predict(X, **kwargs)

        return pred

    def evaluate(self, X, y, **kwargs):
        scores = self.model.score(X, y)

        return scores
    
    def save(self, model_file):
        with fs.open(model_file, 'wb') as f:
            pickle.dump(self, f, protocol=4)

    @staticmethod
    def load(model_file):
        with fs.open(model_file, 'rb') as f:
            return pickle.load(f)
    
    def get_iteration_scores(self):
        return {}

In [33]:
class KnnModel(HyperModel):
    def __init__(self, searcher, reward_metric=None, task=None):
        super(KnnModel, self).__init__(searcher, reward_metric=reward_metric, task=task)
    
    def _get_estimator(self, space_sample):
        return KnnEstimator(space_sample, task=self.task)
    
    def load_estimator(self, model_file):
        return KnnEstimator.load(model_file)

In [50]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

X, y = load_digits(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X[:3]

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0


In [55]:
model = KnnModel(searcher=searcher, task='binary', reward_metric='accuracy')
model.search(X_train, y_train, X_test, y_test)

06-24 11:32:46 I hypernets.u.common.py 124 - 10 class detected, inferred as a [multiclass classification] task
06-24 11:32:46 I hypernets.c.meta_learner.py 22 - Initialize Meta Learner: dataset_id:98aca899f062bf957947e497ae4237e3
06-24 11:32:46 I hypernets.d.in_process_dispatcher.py 77 - Trial 1 done, reward: 0.9777777777777777, best_trial_no:1, best_reward:0.9777777777777777

06-24 11:32:46 I hypernets.d.in_process_dispatcher.py 77 - Trial 2 done, reward: 0.9777777777777777, best_trial_no:1, best_reward:0.9777777777777777

06-24 11:32:46 I hypernets.d.in_process_dispatcher.py 77 - Trial 3 done, reward: 0.9861111111111112, best_trial_no:3, best_reward:0.9861111111111112

06-24 11:32:46 I hypernets.d.in_process_dispatcher.py 77 - Trial 4 done, reward: 0.9888888888888889, best_trial_no:4, best_reward:0.9888888888888889

06-24 11:32:46 I hypernets.d.in_process_dispatcher.py 77 - Trial 5 done, reward: 0.9888888888888889, best_trial_no:4, best_reward:0.9888888888888889

06-24 11:32:46 I hyp

In [44]:
best_model = model.get_best_trial()
final_model = model.final_train(best_model.space_sample, X_train, y_train)

In [45]:
final_model.evaluate(X_train, y_train)

0.9895615866388309

In [47]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X1, y1 = load_iris(return_X_y=True, as_frame=True)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
X1[:3]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [57]:
# another dataset, task set as 'abc'
model1 = KnnModel(searcher=searcher, task='abc', reward_metric='accuracy')
model1.search(X1_train, y1_train, X1_test, y1_test)

06-24 11:33:25 I hypernets.u.common.py 124 - 3 class detected, inferred as a [multiclass classification] task
06-24 11:33:25 I hypernets.c.meta_learner.py 22 - Initialize Meta Learner: dataset_id:74e7c134740a0f846f4c5e57fa5e6c93
06-24 11:33:25 I hypernets.d.in_process_dispatcher.py 77 - Trial 1 done, reward: 1.0, best_trial_no:1, best_reward:1.0

06-24 11:33:25 I hypernets.d.in_process_dispatcher.py 77 - Trial 2 done, reward: 1.0, best_trial_no:1, best_reward:1.0

06-24 11:33:25 I hypernets.d.in_process_dispatcher.py 77 - Trial 3 done, reward: 1.0, best_trial_no:1, best_reward:1.0

06-24 11:33:25 I hypernets.d.in_process_dispatcher.py 77 - Trial 4 done, reward: 1.0, best_trial_no:1, best_reward:1.0

06-24 11:33:25 I hypernets.d.in_process_dispatcher.py 77 - Trial 5 done, reward: 1.0, best_trial_no:1, best_reward:1.0

06-24 11:33:25 I hypernets.d.in_process_dispatcher.py 77 - Trial 6 done, reward: 1.0, best_trial_no:1, best_reward:1.0

06-24 11:33:25 I hypernets.d.in_process_dispatcher.

In [58]:
best_model1 = model1.get_best_trial()
final_model1 = model1.final_train(best_model1.space_sample, X1_train, y1_train)
final_model1.evaluate(X1_train, y1_train)

1.0